In [1]:
def symb_perf_stats_vectorized_v1(df_symbols_close):
    """Takes dataframe of symbols' close and returns symbols, period_yr,
       drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI
       https://stackoverflow.com/questions/36750571/calculate-max-draw-down-with-a-vectorized-solution-in-python
       http://www.tangotools.com/ui/ui.htm
       Calculation CHECKED against: http://www.tangotools.com/ui/UlcerIndex.xls
       Calculation VERIFIED in: symb_perf_stats_vectorized.ipynb

    Args:
        df_symbols_close(dataframe): dataframe with date as index,
          symbol's close in columns, and symbols as column names.

    Return:
        symbols(pandas.core.indexes.base.Index): stock symbols
        period_yr(float): years, (days in dataframe) / 252
        drawdown(numpy array): drawdown from peak, 0.05 means 5% drawdown,
            with date index and symbols as column names
        UI(pandas.series float64): ulcer-index
        max_drawdown(pandas series float64): maximum drawdown from peak
        returns_std(pandas series float64): standard deviation of daily returns
        Std_UI(pandas series float64): returns_std / UI
        CAGR(pandas series float64): compounded annual growth rate
        CAGR_Std(pandas series float64): CAGR / returns_std
        CAGR_UI(pandas series float64): CAGR / UI
    """
    # v1 convert drawdown from pandas series to numpy array

    import numpy as np
    import pandas as pd

    symbols = df_symbols_close.columns
    df_symbols_returns = df_symbols_close / df_symbols_close.shift(1) - 1
    # standard deviation divisor is N - ddof
    returns_std = df_symbols_returns.std(ddof=1)

    # +++ SET RETURNS OF FIRST ROW = 0,
    #  otherwise drawdown calculation starts with the second row
    df_symbols_returns.iloc[0] = 0
    cum_returns = (1 + df_symbols_returns).cumprod()
    
    drawdown = cum_returns.div(cum_returns.cummax()) - 1
    # convert from pandas Series into a NumPy array
    drawdown = np.array(drawdown)

    max_drawdown = drawdown.min()

    UI = np.sqrt(np.sum(np.square(drawdown), axis=0) / len(drawdown))    
    UI = pd.Series(UI, index=symbols)

    Std_UI = returns_std / UI
    period_yr = len(df_symbols_close) / 252  # 252 trading days per year
    CAGR = (df_symbols_close.iloc[-1] / df_symbols_close.iloc[0]) \
        ** (1 / period_yr) - 1
    CAGR_Std = CAGR / returns_std
    CAGR_UI = CAGR / UI

    return symbols, period_yr, drawdown, UI, max_drawdown, \
        returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI

In [2]:
import pandas as pd
# from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1
from myUtils import pickle_load, symb_perf_stats_vectorized

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [ ]:
df_c

In [ ]:
# symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    # symb_perf_stats_vectorized(df_c)
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized(df_c)    
caches_perf_stats_vect = []

In [5]:
%%timeit -n 20 -r 17
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized(df_c)
# UI
# # drawdown

92.6 ms ± 4.53 ms per loop (mean ± std. dev. of 17 runs, 20 loops each)


In [6]:
%%timeit -n 20 -r 17
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized_v1(df_c)
# print(0, UI['A'])
# print(1, Std_UI['A'])
# print(2, CAGR['A'])
# print(3, CAGR_Std['A'])
# print(4, CAGR_UI['A'])

83.5 ms ± 3.51 ms per loop (mean ± std. dev. of 17 runs, 20 loops each)


In [ ]:
def perf_eval_0(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [ ]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval_0(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()
# df_perf.tail()

In [ ]:
def perf_eval_1(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized_v1(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      # date_first = drawdown.index[0].strftime('%Y-%m-%d')
      # date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [ ]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval_1(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()

In [ ]:
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized_v1(df_c)
caches_perf_stats_vect = []
# for symbol in symbols:
for symbol in symbols[:3]:
    # date_first = drawdown.index[0].strftime('%Y-%m-%d')
    # date_last = drawdown.index[-1].strftime('%Y-%m-%d')
    date_first = df_c.index[0].strftime('%Y-%m-%d')
    date_last = df_c.index[-1].strftime('%Y-%m-%d')
    # print('1', date_first, date_last)
    # print('2', CAGR[symbol], CAGR)
    # # print(UI[symbol])
    # print('3 +++++++++', UI)    
    # print('4', Std_UI[symbol], Std_UI)
    # print('5', CAGR_Std[symbol], CAGR_Std)
    # print('6', CAGR_UI[symbol], CAGR_UI)

    print('0', symbols[:3])
    print('1', date_first, date_last)
    print('2', CAGR)
    # print(UI[symbol])
    print('3', UI)    
    print('4', Std_UI)
    print('5', CAGR_Std)
    print('6', CAGR_UI)    

    # cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
    #         UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])

In [ ]:
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized(df_c)
caches_perf_stats_vect = []
for symbol in symbols:
    # date_first = drawdown.index[0].strftime('%Y-%m-%d')
    # date_last = drawdown.index[-1].strftime('%Y-%m-%d')
    date_first = df_c.index[0].strftime('%Y-%m-%d')
    date_last = df_c.index[-1].strftime('%Y-%m-%d')
    # print('1', date_first, date_last)
    # print('2', CAGR[symbol], CAGR)
    # # print(UI[symbol])
    # print('3 +++++++++', UI)    
    # print('4', Std_UI[symbol], Std_UI)
    # print('5', CAGR_Std[symbol], CAGR_Std)
    # print('6', CAGR_UI[symbol], CAGR_UI)

    print('1', date_first, date_last)
    print('2', CAGR)
    # print(UI[symbol])
    print('3', UI)    
    print('4', Std_UI)
    print('5', CAGR_Std)
    print('6', CAGR_UI)   

In [ ]:
# _cols2 = ['TSBK', 'ELF', 'AVEO', 'ASC', 'TPL', 'AXON', 'NRIM', 'AAON', 'UFPT', 'GILD', 'MOD', 'SANM', 'INSW', 'LOPE', 'AE', 'ABMD', 'RMBS', 'WNC']
_cols2 = ['AE', 'ABMD', 'RMBS', 'WNC']
df_temp = df_c[_cols2].copy()
# df_perf, grp_CAGRUI_mean_std = perf_eval(df_temp)
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_temp)
# print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

In [ ]:
_df = df_perf.loc[df_perf['symbol'].isin(_cols2)]
_means = _df[['Std/UI',	'CAGR/Std', 'CAGR/UI']].mean()
_stds = _df[['Std/UI',	'CAGR/Std', 'CAGR/UI']].std()
_mean_std = _means / _stds
# print(_means, _means[0], _means[1], _means[2])
# print(_stds, _stds[0], _stds[1], _stds[2])
print(_mean_std, _mean_std[0], _mean_std[1], _mean_std[2])
_df

# df.loc[df['points'].isin([7, 9, 12])]

In [ ]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()